In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from fastai.vision import *
from fastai.metrics import accuracy
from fastai.basic_data import *
from skimage.util import montage
from fastai.callbacks.hooks import num_features_model
from torch.nn import L1Loss
import pandas as pd
from torch import optim
import re
import json
#import cv2
import types
import random

from utils import *

In [2]:
WIDTH = 384
HEIGHT = 128
SZ = 224
BS = 64
NUM_WORKERS = 6

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
root_dir = Path('../input')
train_dir = root_dir/'train'
test_dir = root_dir/'test'
old_train_dir = root_dir/'old/train'
bg_dir = root_dir/'sea'
annot_dir = root_dir/'annotation'
learn_path = Path('../')
crop_dir = root_dir/'crop'

In [4]:
class StubbedObjectCategoryList(ObjectCategoryList):
    def analyze_pred(self, pred): return [pred.unsqueeze(0), torch.ones(1).long()]

In [5]:
df = pd.read_csv(root_dir/'train.csv')
classes = df.Id.unique()
train_imgs = df.Image.tolist()
    
data = (
    ImageItemList
        #.from_folder(train_dir)
        .from_df(df, train_dir, cols=['Image'])
        .no_split()
        #.split_by_valid_func(lambda path: path) # in newer version of the fastai library there is .no_split that could be used here
        #.label_from_func(lambda path: fn2label[path2fn(path)], classes=classes)
        .label_empty()
        .add_test(ImageItemList.from_folder(test_dir))
        .transform(None, size=SZ, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=BS, num_workers=NUM_WORKERS, path=root_dir)
        .normalize(imagenet_stats)
)

You can deactivate this warning by passing `no_check=True`.


/home/wb/study/fastai/fastai/basic_data.py:236: UserWarning: It's not possible to collate samples of your dataset together in a batch.
  warn(message)


In [9]:
len(data.test_dl)

125

In [7]:
data.test_dl

DeviceDataLoader(dl=<torch.utils.data.dataloader.DataLoader object at 0x7fbbdddfbe10>, device=device(type='cpu'), tfms=[functools.partial(<function _normalize_batch at 0x7fbbdfba0268>, mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]), do_x=True, do_y=False)], collate_fn=<function data_collate at 0x7fbbe07c2730>)

In [6]:
data.train_ds.x

ImageItemList (25361 items)
[Image (3, 700, 1050), Image (3, 325, 758), Image (3, 497, 1050), Image (3, 525, 1050), Image (3, 525, 1050)]...
Path: ../input/train

In [66]:
#check if the orders of df and train_ds are the same
if 0:
    for k, (df_fn, ds_img) in enumerate(zip(train_imgs, data.train_ds.x)):
        df_img = open_image(train_dir/df_fn)
        if (df_img.size != ds_img.size):
            print(df_fn)
        if k % 1000 == 0:
            print(k)
    print('comparing over')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
comparing over


In [7]:
data.show_batch(rows=3, ds_type=DatasetType.Train, figsize=(12,12))

RuntimeError: Traceback (most recent call last):
  File "/home/wb/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 138, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/wb/study/fastai/fastai/torch_core.py", line 115, in data_collate
    return torch.utils.data.dataloader.default_collate(to_data(batch))
  File "/home/wb/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 232, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/home/wb/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 232, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/home/wb/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 209, in default_collate
    return torch.stack(batch, 0, out=out)
RuntimeError: invalid

In [8]:
class FlukeDetector(nn.Module):
    def __init__(self, arch=models.resnet18):
        super().__init__() 
        self.cnn = create_body(arch)
        self.head = create_head(num_features_model(self.cnn) * 2, 4)
        
    def forward(self, im):
        x = self.cnn(im)
        x = self.head(x)
        return 2 * (x.sigmoid_() - 0.5)

In [9]:
def loss_fn(preds, targs, class_idxs):
    return L1Loss()(preds, targs.squeeze())

In [10]:
iou = partial(dice, iou=True)

In [11]:
dice

<function fastai.metrics.dice(input: torch.Tensor, targs: torch.Tensor, iou: bool = False) -> <function NewType.<locals>.new_type at 0x7fdfb566ec80>>

In [12]:
learn = Learner(data, FlukeDetector(arch=models.resnet50), loss_func=loss_fn, path=learn_path)
learn.metrics = [lambda preds, targs, _: IoU(preds, targs.squeeze()).mean()]

In [13]:
learn.load('fluke_location')
learn.model.to(device)

FlukeDetector(
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): Conv2d(64, 256, ker

In [80]:
learn.model.eval()
with torch.no_grad():
    preds = get_preds(learn.model, data.test_dl)
    

RuntimeError: Traceback (most recent call last):
  File "/home/wb/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 138, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/wb/study/fastai/fastai/torch_core.py", line 115, in data_collate
    return torch.utils.data.dataloader.default_collate(to_data(batch))
  File "/home/wb/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 232, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/home/wb/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 232, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/home/wb/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 209, in default_collate
    return torch.stack(batch, 0, out=out)
RuntimeError: invalid

In [ ]:
# preds, targs = learn.get_preds()

# predicted_bboxes = ((preds + 1) / 2 * SZ).numpy()
# targets = ((targs + 1) / 2 * SZ).numpy().squeeze()

# def draw_bbox(img, bbox, target=None, color=(255, 0, 0), thickness=2):
#     y_min, x_min, y_max, x_max = map(int, bbox)
#     cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
#     if target is not None:
#         y_min, x_min, y_max, x_max = map(int, target)
#         cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=(0,255,0), thickness=thickness)
#     return img

# def cv_read(path):    
#     im = cv2.imread(path, cv2.IMREAD_COLOR)
#     return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

# # Targets are in green, predictions in red.
# ims = np.stack([draw_bbox(cv_read(data.valid_ds.items[i]), predicted_bboxes[i], targets[i]) for i in range(9)])
# plt.figure(figsize=(12,12))
# plt.axis('off')
# plt.imshow(montage(np.stack(ims), multichannel=True))